# ANALYZING RTC SEVERITY DATASET
In this project, we are analyzing rioad accident data in order to answer the following questions:
1. When do most road accidents occur? What hour?
2. Which roads lead to major road accidents.
3. Which vehicles lead to fatal or serious accidents?

In [ ]:
import csv
import datetime as dt

In [3]:
# read the csv file
with open("C://Users//user//Downloads//accident_data.csv") as file:
    rows=list(csv.reader(file))

In [6]:
print(rows[:5])

[['Accident_Index', '1st_Road_Class', '1st_Road_Number', '2nd_Road_Class', '2nd_Road_Number', 'Accident_Severity', 'Carriageway_Hazards', 'Date', 'Day_of_Week', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Control', 'Junction_Detail', 'Latitude', 'Light_Conditions', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'LSOA_of_Accident_Location', 'Number_of_Casualties', 'Number_of_Vehicles', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Police_Force', 'Road_Surface_Conditions', 'Road_Type', 'Special_Conditions_at_Site', 'Speed_limit', 'Time', 'Urban_or_Rural_Area', 'Weather_Conditions', 'Year', 'InScotland'], ['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E010

In [7]:
header = rows[0]
print(len(header))

34


In [8]:
data = rows[1:]
print(rows[-5:])

[['201091NM01760', 'A', '95', '', '0', 'Slight', 'None', '18/02/2010', 'Thursday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '57.374005', 'Daylight', 'Highland', 'Highland', '311830', '832450', '-3.467828', '', '2', '1', '0', '0', 'Northern', 'Dry', 'Single carriageway', 'None', '60', '07:00', 'Rural', 'Fine no high winds', '2010', 'Yes'], ['201091NM01881', 'A', '95', '', '0', 'Slight', 'None', '21/02/2010', 'Sunday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '57.232273', 'Darkness - no lighting', 'Highland', 'Highland', '290880', '817170', '-3.809281', '', '1', '1', '0', '0', 'Northern', 'Frost or ice', 'Single carriageway', 'None', '60', '03:00', 'Rural', 'Fine no high winds', '2010', 'Yes'], ['201091NM01935', 'A', '96', 'Unclassified', '0', 'Slight', 'None', '23/02/2010', 'Tuesday', '1', 'Give way or uncontrolled', 'T or staggered junction', '57.585044', 'Daylight', 'Highland', 'Highland', '288730', '856520', '-3.86